In [1]:
import os
import cv2
import numpy as np
import functools

from tensorpack.dataflow import MultiProcessMapDataZMQ,MultiProcessMapAndBatchDataZMQ, TestDataSpeed
from tensorpack.dataflow.common import MapData

import sys
sys.path.append('../')

from dataset.augmentors import CropAug, FlipAug, ScaleAug, RotateAug, ResizeAug
from dataset.base_dataflow import CocoDataFlow, JointsLoader
from dataset.dataflow_steps import create_all_mask, augment, read_img, apply_mask, gen_mask
from dataset.label_maps import PredictionData

def build_sample(components):
    """
    Builds a sample for a model.

    :param components: components
    :return: list of final components of a sample.
    """
    pred_data = PredictionData(components[13])

    return [components[10], pred_data.kp_heatmaps(),
            pred_data.compute_short_offsets(), pred_data.compute_mid_offsets()]


def get_dataflow(annot_path, img_dir, strict, x_size = 224):
    """
    This function initializes the tensorpack dataflow and serves generator
    for training operation.

    :param annot_path: path to the annotation file
    :param img_dir: path to the images
    :return: dataflow object
    """
    coco_crop_size = 368

    # configure augmentors

    augmentors = [
        ScaleAug(scale_min=0.8,
                 scale_max=2.0,
                 target_dist=0.8,
                 interp=cv2.INTER_CUBIC),

        RotateAug(rotate_max_deg=30,
                  interp=cv2.INTER_CUBIC,
                  border=cv2.BORDER_CONSTANT,
                  border_value=(128, 128, 128), mask_border_val=1),

        #CropAug(coco_crop_size, coco_crop_size, center_perterb_max=40, border_value=128,
        #        mask_border_val=1),

        FlipAug(num_parts=17, prob=0.5),

        ResizeAug(x_size[1], x_size[0])

    ]

    # prepare augment function

    augment_func = functools.partial(augment,
                                     augmentors=augmentors)

    # prepare building sample function

    build_sample_func = functools.partial(build_sample)

    df = CocoDataFlow((coco_crop_size, coco_crop_size), annot_path, img_dir)
    df.prepare()
    size = df.size()
    df = MapData(df, read_img)
    df = MapData(df, augment_func)
    df = MultiProcessMapAndBatchDataZMQ(df, num_proc=4, map_func=build_sample_func,batch_size=4)

    return df, size

2023-03-30 17:22:14.788155: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 17:22:14.929241: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-30 17:22:15.728734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sathish/miniconda3/envs/tf/lib/python3.9/site-packages/cv2/../../lib64:/opt/ros/humble/opt/r

In [2]:
batch_size = 10
curr_dir = os.path.dirname("__file__")
annot_path = os.path.join(curr_dir, '../../data/coco2017/annotations/person_keypoints_val2017.json')
img_dir = os.path.abspath(os.path.join(curr_dir, '../../data/coco2017/val2017/'))

df1, size1 = get_dataflow(annot_path, img_dir, False, x_size=(353,281))
TestDataSpeed(df1).start()

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
Loading image annot 0/5000
Loading image annot 2000/5000
Loading image annot 4000/5000
[0330 17:22:21 @argtools.py:146] WRN Starting a process with 'fork' method is not safe and may consume unnecessary extra CPU memory. Use 'forkserver/spawn' method (available after Py3.4) instead if you run into any issues. See https://docs.python.org/3/library/multiprocessing.html#contexts-and-start-methods


  1%|#4                                                                                                                 |61/5000[01:33<2:06:01, 0.65it/s]


KeyboardInterrupt: 